In [1]:
import numpy as np 
import matplotlib as plt 
import os 
import time
import pandas as pd
#import tensorflowjs as tfjs
#from sklearn.model_selection import train_test_split
from tensorflow.keras import layers,models,activations
from tensorflow.keras.utils import to_categorical 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Input,LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

## Load and Preprocess Data

In [2]:
DATA_PATH = os.path.DATA_PATH = os.path.join(os.getcwd(),"Fingerpose_data")
gestures = ['play','pause','nothing']
curl_map = {'No Curl':0,'Half Curl':1,'Full Curl':2}
direction_map = {'Vertical Up':0,'Vertical Down':1,'Horizontal Left':2,'Horizontal Right':3,'Diagonal Up Right':4,
                 'Diagonal Up Left':5,'Diagonal Down Right':6,'Diagonal Down Left':7}
label_map = {label:num for num, label in enumerate(gestures)}

In [3]:
#read data from CSV file and save to a dataset 
def load_csv(data,gesture,dataset,labels):
    data = data.reset_index()
    frame,datapoints = [],[]
    for index, row in data.iterrows():
        frame.append(curl_map[row['Curl']])
        frame.append(direction_map[row['Direction']])
        if (index+1)%5 == 0:
            datapoints.append(frame)
            frame = []
        if (index+1)%150 == 0:
            dataset.append(datapoints)
            labels.append(label_map[gesture])
            datapoints=[]

In [4]:
dataset,labels = [],[]
for gesture in gestures:
    files = os.listdir(os.path.join(DATA_PATH,gesture))
    for file in files:
        df = pd.read_csv(os.path.join(DATA_PATH,gesture,file))
        load_csv(df,gesture,dataset,labels)
        
            

In [6]:
X = np.array(dataset)
y = to_categorical(labels).astype(int)
print(X.shape)
print(y.shape)
#X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.1)


(60, 30, 10)
(60, 3)


## Train LSTM Model 

In [18]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [19]:
inputs = Input((30,10))
lstm = LSTM(64,return_sequences=True, activation = 'relu',input_shape=(30,10))(inputs)
lstm2 = LSTM(128, return_sequences=True,activation = 'relu')(lstm)
lstm3 = LSTM(64, activation = 'relu')(lstm2)
de = Dense(64, activation = 'relu')(lstm3)
de1 = Dense(128, activation='relu')(de)
de2 = Dense(64, activation='relu')(de1)
outputs = Dense(len(gestures),activation='softmax')(de2)
model = models.Model(inputs,outputs)
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 30, 10)]          0         
                                                                 
 lstm_21 (LSTM)              (None, 30, 64)            19200     
                                                                 
 lstm_22 (LSTM)              (None, 30, 128)           98816     
                                                                 
 lstm_23 (LSTM)              (None, 64)                49408     
                                                                 
 dense_4 (Dense)             (None, 64)                4160      
                                                                 
 dense_5 (Dense)             (None, 128)               8320      
                                                                 
 dense_6 (Dense)             (None, 64)                8256

In [20]:
model.fit(X, y, epochs=100, callbacks=[tb_callback])

Epoch 1/100


2022-09-15 22:13:17.112116: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


2/2 [==============================] - 1s 26ms/step - loss: 1.0848 - categorical_accuracy: 0.3167
Epoch 2/100
2/2 [==============================] - 0s 20ms/step - loss: 1.0924 - categorical_accuracy: 0.3833
Epoch 3/100
2/2 [==============================] - 0s 26ms/step - loss: 1.0620 - categorical_accuracy: 0.3500
Epoch 4/100
2/2 [==============================] - 0s 22ms/step - loss: 1.0240 - categorical_accuracy: 0.3833
Epoch 5/100
2/2 [==============================] - 0s 22ms/step - loss: 0.9912 - categorical_accuracy: 0.5000
Epoch 6/100
2/2 [==============================] - 0s 22ms/step - loss: 0.9025 - categorical_accuracy: 0.6333
Epoch 7/100
2/2 [==============================] - 0s 22ms/step - loss: 0.7290 - categorical_accuracy: 0.6833
Epoch 8/100
2/2 [==============================] - 0s 22ms/step - loss: 0.5978 - categorical_accuracy: 0.7667
Epoch 9/100
2/2 [==============================] - 0s 21ms/step - loss: 0.5190 - categorical_accuracy: 0.8000
Epoch 10/100
2/2 [====

Epoch 75/100
2/2 [==============================] - 0s 26ms/step - loss: 0.0500 - categorical_accuracy: 1.0000
Epoch 76/100
2/2 [==============================] - 0s 25ms/step - loss: 0.0357 - categorical_accuracy: 1.0000
Epoch 77/100
2/2 [==============================] - 0s 24ms/step - loss: 0.0288 - categorical_accuracy: 1.0000
Epoch 78/100
2/2 [==============================] - 0s 25ms/step - loss: 0.0244 - categorical_accuracy: 1.0000
Epoch 79/100
2/2 [==============================] - 0s 25ms/step - loss: 0.0226 - categorical_accuracy: 1.0000
Epoch 80/100
2/2 [==============================] - 0s 25ms/step - loss: 0.0195 - categorical_accuracy: 1.0000
Epoch 81/100
2/2 [==============================] - 0s 24ms/step - loss: 0.0118 - categorical_accuracy: 1.0000
Epoch 82/100
2/2 [==============================] - 0s 24ms/step - loss: 0.0065 - categorical_accuracy: 1.0000
Epoch 83/100
2/2 [==============================] - 0s 24ms/step - loss: 0.0050 - categorical_accuracy: 1.0000
E

In [33]:
#convert and save file in tfjs format 
tfjs.converters.save_keras_model(model,os.getcwd())

In [21]:
#save model in python format
model.save('gestures.h5')